In [1]:
from mujoco_py import load_model_from_path, MjSim, MjViewer
from mujoco_py import MjRenderContextOffscreen
import mujoco_py

from IPython.display import HTML, display
from matplotlib import pyplot as plt
from copy import deepcopy
import numpy as np
import imageio
import base64
import types
import glob
import cv2
import io
import os

In [2]:
MODEL_XML_PATH = os.path.join('fetch', 'pick_and_place.xml')
print(MODEL_XML_PATH)

fetch/pick_and_place.xml


In [3]:
initial_qpos = {
    'robot0:slide0': 0.405,
    'robot0:slide1': 0.48,
    'robot0:slide2': 0.0,
    'object0:joint': [1.23, 0.75, 0.5, 1.0, 0.0, 0.0, 0.0],
}
print(initial_qpos)

{'robot0:slide0': 0.405, 'robot0:slide1': 0.48, 'robot0:slide2': 0.0, 'object0:joint': [1.23, 0.75, 0.5, 1.0, 0.0, 0.0, 0.0]}


In [4]:
model_path = MODEL_XML_PATH
has_object = True
block_gripper = False
n_substeps = 1
gripper_extra_height = 0.2
target_in_the_air = True
target_offset = 0.0
obj_range = 0.15
target_range = 0.15
distance_threshold = 0.05
reward_type = "sparse"

fullpath = 'gym_assets/{}'.format(model_path)
print(fullpath)

model = load_model_from_path(fullpath)
sim = MjSim(model, nsubsteps=n_substeps)
viewer = MjViewer(sim)

gym_assets/fetch/pick_and_place.xml
Creating window glfw


In [6]:
sim.model.eq_data

array([[ 1.4149,  0.2641,  0.786 ,  1.    , -0.    , -0.    , -0.    ]])

In [7]:
sim.model.eq_type

array([1], dtype=int32)

In [5]:
def reset_mocap_welds(sim):
    """Resets the mocap welds that we use for actuation.
    """
    if sim.model.nmocap > 0 and sim.model.eq_data is not None:
        for i in range(sim.model.eq_data.shape[0]):
            if sim.model.eq_type[i] == mujoco_py.const.EQ_WELD:
                sim.model.eq_data[i, :] = np.array(
                    [0., 0., 0., 1., 0., 0., 0.])
    sim.forward()


In [6]:
def save_video(frames, filename='video/mujoco.mp4', fps=60):
    writer = imageio.get_writer(filename, fps=fps)
    for f in frames:
        writer.append_data(f)
    writer.close()
    
def show_video(filname='video/mujoco.mp4'):
    mp4list = glob.glob(filname)
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
        
def show_image(img):
    #cv2.imshow("test", img)
    plt.figure(figsize = (16,9))
    plt.axis('off')
    plt.imshow(img)

In [7]:
arm_joint_list = ['torso_lift_joint', 'head_pan_joint', 'head_tilt_joint', 'shoulder_pan_joint', 'shoulder_lift_joint',\
                  'upperarm_roll_joint', 'elbow_flex_joint',\
                  'forearm_roll_joint', 'wrist_flex_joint', 'wrist_roll_joint']
arm_joint_list = ['robot0:{}'.format(joint_name) for joint_name in arm_joint_list]
gripper_joint_list = ['r_gripper_finger_joint', 'l_gripper_finger_joint']
gripper_joint_list = ['robot0:{}'.format(joint_name) for joint_name in gripper_joint_list]

In [8]:
for name, value in initial_qpos.items():
    sim.data.set_joint_qpos(name, value)
reset_mocap_welds(sim)
sim.forward()
    
# Move end effector into position.
gripper_target = np.array([-0.498, 0.005, -0.431 + gripper_extra_height]) + sim.data.get_site_xpos('robot0:grip')
print(gripper_target)
gripper_rotation = np.array([1., 0., 1., 0.])
sim.data.set_mocap_pos('robot0:mocap', gripper_target)
sim.data.set_mocap_quat('robot0:mocap', gripper_rotation)
for _ in range(10):
    sim.step()

# Extract information for sampling goals.
initial_gripper_xpos = sim.data.get_site_xpos('robot0:grip').copy()
if has_object:
    height_offset = sim.data.get_site_xpos('object0')[2]
print(initial_gripper_xpos)
print(height_offset)

[1.3419 0.7491 0.555 ]
[1.78386683 0.74495894 0.71920402]
0.4982342647440577


In [9]:
lookat_refer = sim.data.get_body_xpos('object0')
viewer.cam.lookat[0] = lookat_refer[0]
viewer.cam.lookat[1] = lookat_refer[1]
viewer.cam.lookat[2] = lookat_refer[2]
viewer.cam.azimuth = 150
viewer.cam.elevation = -30
viewer.cam.distance = 1.5

In [10]:
gripper_ctrl = 0.05

frames = []
cnt = 0
steps = 100
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('object0') + np.array([0, 0, 0.1])
    sim.data.mocap_quat[0] = np.array([1, 0, 1, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
show_video('video/mujoco.mp4')

In [11]:
gripper_ctrl = 0.05

frames = []
cnt = 0
steps = 100
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('object0') + np.array([0, 0, 0.02])
    sim.data.mocap_quat[0] = np.array([1, 0, 1, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
show_video('video/mujoco.mp4')

In [12]:
for i in range(sim.data.ncon):
    con = sim.data.contact[i]
    print(con.dist, sim.model.geom_id2name(con.geom1), sim.model.geom_id2name(con.geom2))

1.0895756314410754e-06 floor0 robot0:base_link
1.091545917791814e-06 floor0 robot0:base_link
1.1003680197851917e-06 floor0 robot0:base_link
-0.0001986561981228048 None object0
-0.00019863749174879884 None object0
-0.00019861982461789293 None object0
-0.00019860111824388698 None object0


In [13]:
gripper_ctrl = -0.05

frames = []
cnt = 0
steps = 100
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('object0') + np.array([0, 0, 0.02])
    sim.data.mocap_quat[0] = np.array([1, 0, 1, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
show_video('video/mujoco.mp4')

In [14]:
gripper_ctrl = -0.05
init_mocap_pos = deepcopy(sim.data.mocap_pos)

frames = []
cnt = 0
steps = 100
for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos + np.array([0, 0, 0.002*j])
    sim.data.mocap_quat[0] = np.array([1, 0, 1, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
show_video('video/mujoco.mp4')